<a href="https://colab.research.google.com/github/mmutyampeta/EduML/blob/main/LASSO_EduML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv("2015_16_DistrictwiseNEW.csv")
print(dataset.shape)

#prepare the data into the right shapes, and inserting mean values in place of NaN values
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(strategy='mean')
dataset = imp_mean.fit_transform(dataset)
dataset = pd.DataFrame(dataset)
X = dataset.loc[:, dataset.columns != 'OVERALL_LI']
y = dataset.iloc[:,11]
X = np.array(X)
y = np.array(y)
y = y.reshape(-1,1)

columns = list(dataset.columns)
features1 = columns[1:11]
features2 = columns[12:]
features = features1 + features2

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

(680, 813)


In [2]:
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
cv_alphas = [0.001,0.003,0.01,0.03,0.1,0.3,1,3,10] #for cross validation in LASSO
model = LassoCV(alphas = cv_alphas)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
score = r2_score(y_test,y_pred)
print(score)

MSE = mean_squared_error(y_test,y_pred)
print(MSE)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3804.0128462236994, tolerance: 6.426739983141742
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1256.227234354653, tolerance: 6.426739983141742
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality 

0.9989206268027174
0.13611890375235708


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18.63718558622532, tolerance: 8.177290152092144
  positive)


In [7]:
from sklearn.inspection import permutation_importance
r = permutation_importance(model, X_test, y_test,
                          n_repeats=30,
                          random_state=0)

best_features = []
for i in r.importances_mean.argsort()[::-1]:
  if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
#    if r.importances_mean[i] > 0.55:
      print(f"{features[i]:<8}"
        f"{r.importances_mean[i]:.3f}"
        f" +/- {r.importances_std[i]:.3f}")
      best_features.append(features[i])

print(best_features)
print(len(best_features))

13      1.847 +/- 0.201
220     0.009 +/- 0.001
495     0.007 +/- 0.001
804     0.005 +/- 0.000
494     0.004 +/- 0.000
6       0.004 +/- 0.000
81      0.004 +/- 0.000
12      0.003 +/- 0.000
480     0.002 +/- 0.000
4       0.002 +/- 0.000
471     0.002 +/- 0.000
16      0.002 +/- 0.000
231     0.002 +/- 0.000
328     0.001 +/- 0.000
453     0.001 +/- 0.000
445     0.001 +/- 0.000
370     0.001 +/- 0.000
325     0.001 +/- 0.000
427     0.001 +/- 0.000
798     0.001 +/- 0.000
361     0.001 +/- 0.000
360     0.001 +/- 0.000
258     0.001 +/- 0.000
17      0.001 +/- 0.000
104     0.001 +/- 0.000
284     0.001 +/- 0.000
245     0.001 +/- 0.000
792     0.001 +/- 0.000
585     0.001 +/- 0.000
640     0.001 +/- 0.000
317     0.001 +/- 0.000
790     0.001 +/- 0.000
117     0.001 +/- 0.000
90      0.001 +/- 0.000
318     0.001 +/- 0.000
429     0.001 +/- 0.000
797     0.001 +/- 0.000
205     0.001 +/- 0.000
365     0.001 +/- 0.000
799     0.001 +/- 0.000
436     0.001 +/- 0.000
441     0.001 +/

IndexError: ignored

In [4]:
BestVar = dataset.loc[:,best_features]
print(BestVar.shape)
X_trainN, X_testN, y_trainN, y_testN = train_test_split(BestVar, y, test_size=0.2, random_state=0) 

(680, 1)


In [6]:
modelNew = LassoCV(alphas = cv_alphas)
modelNew.fit(X_trainN,y_trainN)
y_predN = modelNew.predict(X_testN)
scoreN = r2_score(y_testN,y_predN)
print(scoreN)

MSE = mean_squared_error(y_test,y_pred)
print(MSE)

0.017544757272702438
0.13611890375235708


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:1088: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Finding the coefficients of most importance

In [8]:
for i in model.coef_:
  print(i)

0.0
-8.36179741322841e-05
-0.0001907764334360677
2.891312505728109e-07
-2.963843729026288e-05
-2.972724273779533e-06
-0.0
0.0004876534578310551
-0.0
0.0007111544188199109
0.04378436288990484
0.9658767915569232
-0.005203735967960871
-1.8326354413994675e-06
2.600059165724407e-06
-3.039140716188084e-06
2.9680609130336097e-05
3.0081065751977664e-06
0.0
-0.00020164265728088595
-0.0
0.0
1.930781753723397e-05
0.0
4.229495074222749e-05
-3.4845604035346585e-05
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
-3.361249336152738e-06
0.0002519986795336664
0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0
1.5233501937069587e-05
0.0004586100437020262
-0.0
-0.0
-0.0
-0.0
0.0
0.0
0.0
6.263645164715066e-05
1.1373080982368373e-05
1.2535520166591322e-05
0.0
-1.0809090956104713e-05
-0.0
0.0
5.476085122506965e-05
0.0
-3.516009611305473e-05
-0.00019749184735744915
5.3489106235056114e-05
0.0
0.00013101413089955144
-0.0
0.0
-0.0004113452627420502
0.0
7.70786796569273e-06
-0.0009452959920100367
0.0
-0.0
-0.0
0.0
0.0
-0.0
0.0
-1.5357722837291747e